In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [3]:
from sklearn import preprocessing

In [4]:
def labelencoder(ds):
    encoded_ds=ds.copy()
    le = preprocessing.LabelEncoder()
    for col in encoded_ds.select_dtypes(include=['object']).columns:
        encoded_ds[col]=le.fit_transform(encoded_ds[col])
    
    return encoded_ds

In [5]:
def encoding_for_anomalous(ds):
    encoded_ds=ds.copy()
    for i in range(len(ds)):
        if encoded_ds.iloc[i]=='BENIGN':
            encoded_ds.iloc[i]= 0
        else:
            encoded_ds.iloc[i]= 1
            
    return encoded_ds  

In [6]:
def scaling(ds):
    scaler=preprocessing.StandardScaler()
    scaled_ds=scaler.fit_transform(ds)
    return scaled_ds

In [7]:
def drop_infs(ds):
    ds.replace([np.inf, -np.inf], np.nan, inplace=True)
    ds.dropna(how='any', inplace=True)
    return ds

In [8]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [9]:
PATH="/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv"

In [10]:
normal_ds=pd.read_csv(PATH) #Bu veri setini tamamen train için kullanıcaz

In [11]:
normal_ds=drop_infs(normal_ds)
normal_ds=labelencoder(normal_ds)

In [12]:
normal_y_train=normal_ds[' Label']

In [13]:
normal_x_train=normal_ds.drop(' Label', axis=1) #Label'ı yani y'yi dropluyoruz

In [14]:
normal_x_train=scaling(normal_x_train)

In [15]:
test_path="/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"

In [16]:
test_ds=pd.read_csv(test_path)

In [17]:
test_ds=drop_infs(test_ds)

In [18]:
y_test=test_ds[' Label']

In [19]:
x_test=test_ds.drop(' Label', axis=1)

In [20]:
y_test=encoding_for_anomalous(y_test)
x_test=labelencoder(x_test)
x_test=scaling(x_test)

In [21]:
y_test.value_counts()

1    128025
0     97686
Name:  Label, dtype: int64

In [22]:
#train.head()

In [23]:
#normal_ds.info()

In [24]:
#normal_ds.describe(include='all')

In [25]:
normal_x_train = np.asarray(normal_x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

normal_x_train = tf.cast(normal_x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

In [26]:
input = tf.keras.layers.Input(shape=(78,))

In [27]:
encoder = tf.keras.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu')])(input)


In [28]:
decoder = tf.keras.Sequential([
    layers.Dense(16, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(78, activation='sigmoid')])(encoder)

In [29]:
autoencoder = tf.keras.Model(inputs=input, outputs=decoder)


In [30]:
autoencoder.compile(optimizer='adam', loss='mae')


In [31]:
history = autoencoder.fit(normal_x_train, normal_x_train, 
          epochs=10, 
          batch_size=64,
          validation_data=(x_test, x_test),
          shuffle=True)

2022-12-07 12:36:44.506747: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-07 12:36:44.506921: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


Epoch 1/10
8274/8274 [==============================] - 5s 567us/step - loss: 0.3315 - val_loss: 0.3841
Epoch 2/10
8274/8274 [==============================] - 4s 534us/step - loss: 0.2926 - val_loss: 0.3836
Epoch 3/10
8274/8274 [==============================] - 4s 537us/step - loss: 0.2885 - val_loss: 0.3811
Epoch 4/10
8274/8274 [==============================] - 4s 531us/step - loss: 0.2872 - val_loss: 0.3783
Epoch 5/10
8274/8274 [==============================] - 4s 535us/step - loss: 0.2869 - val_loss: 0.3770
Epoch 6/10
8274/8274 [==============================] - 4s 532us/step - loss: 0.2873 - val_loss: 0.3762ETA: 0s -
Epoch 7/10
8274/8274 [==============================] - 4s 540us/step - loss: 0.2867 - val_loss: 0.3769 ETA: 0s
Epoch 8/10
8274/8274 [==============================] - 4s 533us/step - loss: 0.2861 - val_loss: 0.3764
Epoch 9/10
8274/8274 [==============================] - 4s 540us/step - loss: 0.2862 - val_loss: 0.3756
Epoch 10/10
8274/8274 [========================